---
<div align="center">

# Connect-4 [Artificial Intelligence Project]
</div>

---

<div align="center">

## Search Problem 
</div>

A Search Problem is composed by multiple factors:
- States (Any possible configuration of the Board)
- Initial and Goal States
- Action Space (actions and their effects on the environment)
- Action Cost

---

In [1]:
import numpy as np
from copy import (deepcopy)
from os import (system)
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Connect4:
    # Defining Board's Size
    NROWS = 6
    NCOLS = 7
    
    def __init__(self):
        # Matrix to Store the Board's Values
        self.board = np.zeros(shape=(self.NROWS, self.NCOLS), dtype=np.int8)

        # Array to keep track of the row's index for every column where a piece can be placed
        # Basically it's the Row idx for every column's height
        self.columns_height = np.full(shape=self.NCOLS, fill_value=(self.NROWS - 1), dtype=np.int8)

        # Defining the Possible Actions (Initially a piece can be placed in any column)
        self.actions = set()
        [self.actions.add(col) for col in range(self.NCOLS)]
        
        # Initializing a variable to track the current player
        self.player = 1
        
        # Variable to store the Winner (2 - Game still running || 0 - Tie || 1 - PLayer 1 || -1 - Player 2 / AI)
        self.winner = 2
    
    def move(self, ncol):
        # Creating a new state
        new_state = deepcopy(self)

        # Checks of the given column can be played on
        if(ncol in self.actions):
            # Inserting the move into the board
            new_state.board[new_state.columns_height[ncol]][ncol] = new_state.player
            
            # Updating the "ncol"'s height
            new_state.columns_height[ncol] -= 1
            
            # Checking if the column is full and therefore uncapable of receiving more pieces -> Changes the action space
            if (new_state.columns_height[ncol] < 0):
                new_state.actions.remove(ncol)

            # Updates the current Winner
            new_state.update_winner()
                
            # Updating current player for the next state
            new_state.player *= -1

        # Returns the New State (Could be the same as the previous if the action could not be performed)
        return new_state

    """ MIGHT NEED TO BE OPTIMIZED """
    def get_winner(self, currPiece, Directions:list[tuple]):
        for i in range(0, 6):
            for j in range(0, 7):
                if (self.board[i][j] == currPiece):
                    for (x, y) in Directions:
                        row, col = (i, j)
                        winCondition = 1
                        while(winCondition < 4):
                            row += x
                            col += y
                            if (row in range(0, 6) and col in range(0, 7) and self.board[row][col] == currPiece):
                                winCondition += 1
                            else:
                                break
                        if (winCondition == 4):
                            self.winner = currPiece
                            return

    """ MIGHT NEED TO BE OPTIMIZED """
    def update_winner(self):
        Directions = [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]

        self.get_winner(1, Directions)
        if (self.winner == currPiece):
            return
            
        self.get_winner(-1, Directions)
        if (self.winner == currPiece):
            return
        
        self.winner = 0
    
    def is_over(self):
        # If the Winner corresponds to 2 then the game is not finished otherwise it is
        return self.winner != 2

    # IT ONLY WORKS WITHIN A SINGLE INSTANCE OF THE GAME - WE WANT IT TO BE ABLE TO PLAY WITHIN MULTIPLE INSTANCES / STATES OF IT
    def run(self): # CAN ONLY BE USED IN PVP
        # Creating the Game Loop inside the Terminal
        while not self.is_over():
            system('cls')
            print("\nCURRENT BOARD: \n")
            print(self.board)
            ncol = int(input(f"\n| Player {self.player} | Choose a Column to Play: "))
            self = self.move(ncol)
        
        if self.winner == 0:
            print("Tie")
        else:
            print(f"Player {self.player} Wins!")
    
    def __str__(self):
        CODE = {0:'-', 1:'X', 2:'O'} # Maybe helpfull to convert the board into the same style used in the Assignment 1 Paper
        # new_board = [[[CODE[self.board[row][col]] for row in range(self.NROWS)]] for col in range(self.NCOLS)]
        # return str(new_board)
        return str(self.board)

    def __hash__(self):
        return hash(str(self.board))

    def __eq__(self, other:object):
        if (not isinstance(other, Connect4)):
            return False
        return hash(self) == hash(other)
        

In [3]:
game = Connect4()
print(game)

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]


In [4]:
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
print(game)

[[ 0 -1  0  0  0  0  0]
 [ 0  1  0  0  0  0  0]
 [ 0 -1  0  0  0  0  0]
 [ 0  1  0  0  0  0  0]
 [ 0 -1  0  0  0  0  0]
 [ 0  1  0  0  0  0  0]]


In [5]:
new_game = Connect4()
# new_game.run()

---
<div align="center">

## TreeNode Class
</div>

---

In [6]:
class TreeNode:
    # Screen Parameters
    SQSIZE = 80
    X_OFFSET = 60
    Y_OFFSET = 60
    
    # Circle - Screen Parameters
    CIRCLE_OFFSET = 10
    CIRCLE_POS = (X_OFFSET + SQSIZE//2, Y_OFFSET + SQSIZE//2)
    CIRCLE_RADIUS = (SQSIZE//2) - CIRCLE_OFFSET

    # RGB Colors
    BLACK = (0, 0, 0)
    WHITE = (255, 255, 255)
    BLUE = (135, 206, 250)
    DARK_BLUE = (0, 0, 255)
    RED = (255, 0, 0)

    # Defining a Array with the Piece's Colors [Tuple: (BORDER, INNER CIRCLE)]
    PIECES_COLORS = [(BLACK, WHITE), # Empty Pieces
                     (BLACK, RED), # Player 1
                     (BLACK, BLACK)] # Player 2 / AI
        
    def __init__(self, state, parent=None):
        # Stores a State of the Game
        self.state = state

        # Keeps a reference to his Parent Node
        self.parent = parent

        # Stores all the generated nodes
        self.children = []

    def draw_board(self, screen):
        # Draws Board's Shadow
        board_rect_shadow = pygame.Rect((self.X_OFFSET - 5, self.Y_OFFSET - 5), (self.SQSIZE*game.NCOLS + 2*5, self.SQSIZE*game.NROWS + 2*5))
        pygame.draw.rect(screen, self.BLACK, board_rect_shadow)

        # Draws Main Board
        board_rect = pygame.Rect((self.X_OFFSET, self.Y_OFFSET), (self.SQSIZE*game.NCOLS, self.SQSIZE*game.NROWS))
        pygame.draw.rect(screen, self.DARK_BLUE, board_rect)

        # Drawing Circles in the Board
        for row in range(self.state.NROWS):
            for col in range(self.state.NCOLS):
                # Getting the Colors from the Auxiliar List
                (Border_Color, Circle_Color) = self.PIECES_COLORS[self.state.board[row][col]]

                # Drawing the Circle's Border
                pygame.draw.circle(screen, Border_Color, (self.X_OFFSET + self.SQSIZE//2 + col*(self.SQSIZE),
                                                          self.Y_OFFSET + self.SQSIZE//2 + row*(self.SQSIZE)), self.CIRCLE_RADIUS)

                # Drawing the Main Circle
                pygame.draw.circle(screen, Circle_Color, (self.X_OFFSET + self.SQSIZE//2 + col*(self.SQSIZE),
                                                          self.Y_OFFSET + self.SQSIZE//2 + row*(self.SQSIZE)), int(0.9*self.CIRCLE_RADIUS))
    
    def __hash__(self):
        return hash((self.state, self.parent, [child for child in self.children]))
    
    def __eq__(self, other:object):
        if (not isinstance(other, TreeNode)):
            return False
        return self.__hash__() == other.__hash__()

    def __str__(self):
        return str(self.state)

In [7]:
initial_state = Connect4()
root = TreeNode(initial_state)

---
<div align="center">

## Algorithms
</div>

---

In [8]:
# IMPLEMENT SEARCH ALGORITHMS

def A_Star_Search():
    """ A* Search """
    pass

def MCTS():
    """ Monte Carlo Tree Search """
    pass

---
<div align="center">

## Graphical User Interface
</div>

---

In [18]:
class GUI:
    def __init__(self):
        self.root = TreeNode(state=Connect4())
        self.current_node = deepcopy(self.root)
        self.WIDTH = self.root.state.NCOLS*self.root.SQSIZE + 2*self.root.X_OFFSET
        self.HEIGHT = self.root.state.NROWS*self.root.SQSIZE + 2*self.root.Y_OFFSET

    def draw(self, screen):
        # Filling the Background with Blue
        screen.fill(self.root.BLUE)

        # Drawing the Board Elements
        self.root.draw_board(screen)
    
    def run(self):
        # Initializing Window / Screen
        screen = pygame.display.set_mode((self.WIDTH, self.HEIGHT))
        pygame.display.set_caption("Connect-4")
        ICON_IMG = pygame.image.load('Assets/Connect-Four.png').convert_alpha()
        pygame.display.set_icon(ICON_IMG)

        # Create a Flag to keep track of current state of the Application / GUI
        run = True

        # Main Loop
        while run:

            # Draws the Game Elements into the Screen
            self.draw(screen)
            
            # Event Loop
            for event in pygame.event.get():
                if (event.type == pygame.QUIT):
                    run = False

            # Updates the Window
            pygame.display.update()
        pygame.quit()

In [19]:
App = GUI()

In [20]:
App.run()